Urban Data Science & Smart Cities <br>
URSP688Y Spring 2025<br>
Instructor: Chester Harvey <br>
Urban Studies & Planning <br>
National Center for Smart Growth <br>
University of Maryland

# Demo 8 - Network Analysis

Today, I'm introducing network analysis with a Python package called [OSMnx](https://osmnx.readthedocs.io/en/stable/) that both downloads and analyzes street networks from OpenStreetMap (OSM). This package was written by Geoff Boeing, now at USC, while he was in grad school, a demonstration of how grad students with an interest in coding can author open-source tools with influence that stretches far beyond their own work.

Beneath the hood, OSMnx relies on a network analysis package called [NetworkX](https://networkx.org/); it's one of the dominant Python tools for storing and analyzing networks, or as mathematicians tend to call them, "graphs."

A network or graph, I'll use these terms interchangeably, is made up of "nodes" or "vertices"—the objects that are related to one another—and "edges" or "links"—connections that define relationships between nodes. In a street network, you can think of the nodes as intersections and the edges as street segments. You could also, however, use a graph to represent more abstract concepts, such as communities. Social network models, for example, use nodes to represent people and edges to represent relationships between them. You could imagine adding attributes to those edges to represent the strength of different relationship (e.g., strong or weak ties), just as length, speed limit, or lane counts along edges in a street network are indicators of connection between two intersections.

The following graph has 6 nodes and edges making direct connections between certain pairs of nodes but not others.

<img alt="basic graph" width=300 src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/5b/6n-graf.svg/1920px-6n-graf.svg.png">

We're going to practice analyzing a street network today in terms of shortest paths, a common type of analysis in transportation planning. Another common way of analyzing graphs is based on "degree," the number of nodes connected to a given node. A graph where all nodes are connected to each other would have high "degree centrality," while a graph where nodes are not very well-connected would have low centrality.

Let's install OSMnx, which will automatically install NetworkX as a dependency.

We'll first build a basic graph in NetworkX to demonstrate how it works. Then we'll use OSMnx to download an OSM street network and calculate shortest paths across it.

In [ ]:
# Install OSMNX
# !pip install osmnx

In [ ]:
import os
import pandas as pd
import geopandas as gpd
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt

# Build a Basic Graph

Can we build the same graph pictured here in NetworkX?

<img alt="basic graph" width=300 src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/5b/6n-graf.svg/1920px-6n-graf.svg.png">

In [ ]:
# Create an empty graph
G = nx.Graph()

# Add nodes to the graph
# ! use G.add_node(node_id) to add nodes

# Add edges between nodes
# ! use G.add_edge(a, b) to add edges

# Draw the graph with labels
nx.draw(G, with_labels=True)
plt.show()

In [ ]:
# Degrees from a given node
# ! G.degree(node_id)

In [ ]:
# Centrality for each vertex
# (degrees / count of other nodes)
# ! nx.degree_centrality(G)

In [ ]:
# Average centrality
# ! calculate average as the sum of degree centralities divided by the number of nodes

In [ ]:
# Shortest path
# ! nx.shortest_path(G, a, b)

# Analyzing a Street Network

## Load a street network from OSM

In [ ]:
# Define coordinate systems
UTM18 = 26918
WGS84 = 4326

In [ ]:
# Retrieve the street network for Washington, DC
place = 'Washington, DC, USA'
dc_network_g = ox.graph_from_place(place, network_type='drive')
dc_network_g = ox.project_graph(dc_network_g, to_crs=UTM18)
# Convert to geodataframes for easy plotting and exploration
dc_network_nodes, dc_network_edges = ox.graph_to_gdfs(dc_network_g)

## Load other data

In [ ]:
# Load affordable housing points
affordable_housing = gpd.read_file('Affordable_Housing.geojson').to_crs(UTM18)

In [ ]:
# Load Metro Center point
metro_center = gpd.points_from_xy([-77.032774], [38.8985198])
metro_center = gpd.GeoDataFrame(geometry=metro_center, crs=WGS84).to_crs(epsg=UTM18)

In [ ]:
# Map data to make sure everything lines up
ax = dc_network_edges.plot(color='black', zorder=1)
affordable_housing.plot(ax=ax, color='blue', zorder=2)
metro_center.plot(ax=ax, color='red', zorder=3)

## Relate points to street network

In [ ]:
# Find the nearest node to metro center
metro_center_nodes, metro_center_node_dists = ox.nearest_nodes(
    dc_network_g,
    metro_center.geometry.get_coordinates().x, 
    metro_center.geometry.get_coordinates().y, 
    return_dist=True)

In [ ]:
metro_center_nodes

In [ ]:
metro_center_node_dists

In [ ]:
# Find the nearest nodes to affordable housing units
affordable_housing_nodes, affordable_housing_node_dists = ox.nearest_nodes(
    dc_network_g,
    affordable_housing.geometry.get_coordinates().x,
    affordable_housing.geometry.get_coordinates().y,
    return_dist=True)

In [ ]:
affordable_housing_nodes[:5]

In [ ]:
affordable_housing_node_dists[:5]

## Calculate shortest path

In [ ]:
# Calculate shortest path
affordable_housing_node = affordable_housing_nodes[0]
metro_center_node = metro_center_nodes[0]

route = ox.shortest_path(
    dc_network_g, 
    affordable_housing_node, 
    metro_center_node, 
    weight='length',
)

In [ ]:
fig, ax = ox.plot_graph_route(dc_network_g, route, route_color="y", route_linewidth=6, node_size=0)

## Calculate travel time

In [ ]:
# dc_network_edges.head()

In [ ]:
# impute speed on all edges missing data
dc_network_g = ox.add_edge_speeds(dc_network_g)

# calculate travel time (seconds) for all edges
dc_network_g = ox.add_edge_travel_times(dc_network_g)

# Convert to geodataframes
dc_network_nodes, dc_network_edges = ox.graph_to_gdfs(dc_network_g)

In [ ]:
route = ox.shortest_path(
    dc_network_g, 
    affordable_housing_node, 
    metro_center_node, 
    weight='travel_time',
)

In [ ]:
fig, ax = ox.plot_graph_route(dc_network_g, route, route_color="y", route_linewidth=6, node_size=0)

In [ ]:
# Add up travel time and distance along the route
ox.routing.route_to_gdf(dc_network_g, route)['travel_time'].sum()

In [ ]:
ox.routing.route_to_gdf(dc_network_g, route)['length'].sum()

## Loop calculations for bulk processing

In [ ]:
def shortest_paths_to_metro_center(graph, o_nodes, d_node, weight='length'):    
    # Calculate shortest paths between each O-D pair
    d_nodes = [d_node] * len(o_nodes)
    routes = ox.shortest_path(graph, o_nodes, d_nodes, weight=weight)
    # Gather data for edges along each route
    combined_route_edges = []
    for route_id, route in enumerate(routes):
        route_edges = ox.routing.route_to_gdf(dc_network_g, route)
        route_edges['route_id'] = route_id
        combined_route_edges.append(route_edges)
    combined_route_edges = pd.concat(combined_route_edges, axis=0)
    # Sum length and travel time for edges involved with each route
    route_summaries = combined_route_edges.groupby('route_id')[['length','travel_time']].sum()
    # Clean up column names
    route_summaries = route_summaries.rename(columns={'length':'dist_to_metro_center', 'travel_time': 'time_to_metro_center'})
    return route_summaries
        
routes = shortest_paths_to_metro_center(
    dc_network_g, 
    affordable_housing_nodes, 
    metro_center_node, 
    weight='length'
)

routes.head()

In [ ]:
# Add distance and time estimates back to affordable housing df
affordable_housing = pd.concat([affordable_housing, routes], axis=1)

In [ ]:
affordable_housing.head()

## Compare to straight line distance

In [ ]:
affordable_housing['straight_dist_to_metro_center'] = affordable_housing.distance(metro_center.geometry.iloc[0])

In [ ]:
(affordable_housing['dist_to_metro_center'] - affordable_housing['straight_dist_to_metro_center']).hist()